In [ ]:
import pickle
with open('./data/all_data.pkl', 'rb') as f:
    loaded_data = pickle.load(f)
#print(loaded_data) 
print(loaded_data[0])


In [ ]:
# 리스트 불러오기
import pickle
import json

from FlagEmbedding import FlagLLMReranker

def reranking(data):

    #reranker = FlagLLMReranker('BAAI/bge-reranker-v2-gemma', use_fp16=True)
    reranker = FlagLLMReranker('Qwen/Qwen3-Reranker-8B', use_fp16=True, batch_size=1, max_length=4096)

    message = data["message"]
    references = data["references"]
    pairs = [[message, doc['content']] for doc in references]
    rerank_scores = reranker.compute_score(pairs)

    print("*"*50)
    print(pairs)
    print(references)
    print(rerank_scores)


    results = list(zip(references, rerank_scores))
    results.sort(key=lambda x: x[1], reverse=True) 
    final_result = results[:3]
    print(f"final_result={final_result}")

    data['topk'] = [ doc['docid'] for doc, socre in final_result]
    data["references"] = [ {"score": float(score), "content": doc['content']} for doc, score in final_result ]

    return data


with open("./data/sample_submission.csv", "w") as of:
        idx = 0
        for data in loaded_data:
            print(f"data={data}")
            if len(data["topk"]) > 0:
                response = reranking(data)

            print(f'response: {response}\n')

            # 대회 score 계산은 topk 정보를 사용, answer 정보는 LLM을 통한 자동평가시 활용
            output = {"eval_id": response["eval_id"], "standalone_query": response["standalone_query"], "topk": response["topk"], "answer": response["answer"], "references": response["references"]}
            print(output)
            of.write(f'{json.dumps(output, ensure_ascii=False)}\n')
            idx += 1     